In [ ]:
# Passo 1 e 2 da Instrução:

# Importar a biblioteca pandas para manipulação e análise de dados
import pandas as pd
# Importar numpy para funcionalidades numéricas, como np.nan para representar valores ausentes, se necessário
import numpy as np

# --- Definição dos Caminhos dos Arquivos ---
# Especificar os caminhos completos para os arquivos Excel que serão utilizados.
file_path_data_principal_wms = 'C:/Users/Inteli/Desktop/wms_da_textbook.xlsx'
file_path_dicionario_variaveis_wms = 'C:/Users/Inteli/Desktop/VARIABLES_wms.xlsx'
file_path_dados_limpos_excel = 'C:/Users/Inteli/Desktop/wms_dados_limpos.xlsx' # Novo caminho para o arquivo Excel de saída

# Nomes das abas inferidos dos nomes dos arquivos CSV originais.
sheet_name_data_principal = 'wms_da_textbook' # Aba no arquivo wms_da_textbook.xlsx
sheet_name_dicionario = 'Sheet2'          # Aba no arquivo VARIABLES_wms.xlsx
sheet_name_dados_limpos = 'Dados Limpos'    # Nome da aba para o arquivo Excel de saída

# --- Carregamento e Análise Preliminar do Dicionário de Variáveis ---
# O dicionário de variáveis ajuda a entender o significado de cada coluna no dataset principal.
print("--- Iniciando Carregamento do Dicionário de Variáveis ---")
try:
    # Tentar ler a aba especificada do arquivo Excel do dicionário de variáveis
    df_dicionario_wms = pd.read_excel(file_path_dicionario_variaveis_wms, sheet_name=sheet_name_dicionario)
    # Exibir mensagem de sucesso e informações básicas sobre o dicionário carregado
    print(f"Arquivo do dicionário de variáveis '{file_path_dicionario_variaveis_wms}' (Aba: '{sheet_name_dicionario}') carregado com sucesso.")
    print(f"O dicionário contém {df_dicionario_wms.shape[0]} variáveis e {df_dicionario_wms.shape[1]} colunas descritivas.")

    # Visualizar as primeiras linhas do dicionário de variáveis para inspeção inicial
    print("\nVisualização inicial do dicionário de variáveis (primeiras 5 linhas):\n")
    print(df_dicionario_wms.head().to_string()) # .to_string() para melhor formatação no console

    # Tentar localizar e exibir a descrição da variável 'talent6' no dicionário
    # Supõe-se que a coluna com os nomes das variáveis se chama 'Variable name'
    if 'Variable name' in df_dicionario_wms.columns:
        info_talent6_wms = df_dicionario_wms[df_dicionario_wms['Variable name'] == 'talent6']
        if not info_talent6_wms.empty:
            print("\nInformações sobre a variável 'talent6' encontradas no dicionário:")
            # Usar to_string() para garantir que toda a informação da(s) linha(s) seja impressa
            print(info_talent6_wms.to_string())
        else:
            # Mensagem caso 'talent6' não seja encontrada na coluna esperada
            print("\nAVISO: A variável 'talent6' não foi encontrada na coluna 'Variable name' do dicionário.")
    else:
        # Mensagem caso a coluna 'Variable name' não exista no dicionário
        print("\nAVISO: A coluna 'Variable name' não foi encontrada no dicionário de variáveis.")
        print(f"Colunas disponíveis no dicionário: {df_dicionario_wms.columns.tolist()}")

except FileNotFoundError:
    # Tratamento de erro caso o arquivo do dicionário não seja encontrado
    print(f"\nERRO CRÍTICO: O arquivo do dicionário de variáveis '{file_path_dicionario_variaveis_wms}' não foi encontrado.")
    print("A análise prosseguirá, mas o contexto detalhado das variáveis pode estar ausente.")
    df_dicionario_wms = None # Definir como None para checagens posteriores
except ValueError as ve:
    # Tratamento de erro caso a aba especificada não seja encontrada no arquivo Excel
    print(f"\nERRO: A aba '{sheet_name_dicionario}' não foi encontrada no arquivo '{file_path_dicionario_variaveis_wms}'. Verifique o nome da aba. {ve}")
    df_dicionario_wms = None # Definir como None
except Exception as e:
    # Tratamento para outros erros que possam ocorrer durante o carregamento do dicionário
    print(f"\nERRO: Ocorreu um erro inesperado ao carregar ou processar o dicionário '{file_path_dicionario_variaveis_wms}': {e}")
    df_dicionario_wms = None # Definir como None
print("--- Carregamento do Dicionário de Variáveis Concluído ---")

# --- Carregamento e Limpeza do Dataset Principal (WMS) ---
print("\n--- Iniciando Carregamento e Limpeza do Dataset Principal (WMS) ---")
df_dados_limpos_wms = None # Inicializar para ter um valor caso o try falhe antes da atribuição
try:
    # Carregar a aba especificada do arquivo de dados principal do WMS.
    # O parâmetro 'na_values' ainda é útil com pd.read_excel para strings específicas que devem ser NaN,
    # embora o Excel já tenha sua própria representação de células vazias como NaN.
    df_dados_wms = pd.read_excel(file_path_data_principal_wms, sheet_name=sheet_name_data_principal, na_values=["NA", "N/A", "na", "n/a"])
    # Exibir mensagem de sucesso e dimensões originais do dataset
    print(f"Dataset principal '{file_path_data_principal_wms}' (Aba: '{sheet_name_data_principal}') carregado com sucesso.")
    print(f"Dimensões originais do dataset: {df_dados_wms.shape[0]} linhas, {df_dados_wms.shape[1]} colunas.")

    # Visualizar as primeiras linhas do dataset principal para inspeção
    print("\nVisualização inicial dos dados do WMS (primeiras 5 linhas):\n")
    print(df_dados_wms.head().to_string())

    # Instrução do Passo 2: Lidar com valores ausentes na coluna 'talent6'.
    # Verificar se a coluna 'talent6' existe no DataFrame.
    if 'talent6' in df_dados_wms.columns:
        # Contar e exibir o número de valores ausentes (NA ou NaN) na coluna 'talent6' ANTES da remoção.
        # O método .isnull() retorna True para cada valor ausente, e .sum() conta esses Trues.
        ausentes_antes_talent6_wms = df_dados_wms['talent6'].isnull().sum()
        print(f"\nAnálise da coluna 'talent6':")
        print(f"Número de observações com 'talent6' ausente (NA) ANTES da remoção: {ausentes_antes_talent6_wms}")

        # Remover as linhas (observações) onde a coluna 'talent6' possui valor ausente.
        # O método dropna() com 'subset' especifica a coluna a ser considerada para a remoção.
        # 'inplace=False' (padrão) cria um novo DataFrame, não modificando o original diretamente.
        df_dados_limpos_wms = df_dados_wms.dropna(subset=['talent6'])

        # Contar e exibir o número de valores ausentes em 'talent6' DEPOIS da remoção (deve ser 0).
        ausentes_depois_talent6_wms = df_dados_limpos_wms['talent6'].isnull().sum()
        print(f"Número de observações com 'talent6' ausente (NA) DEPOIS da remoção: {ausentes_depois_talent6_wms}")

        # Exibir as novas dimensões do DataFrame e o número de linhas que foram removidas.
        linhas_removidas_wms = df_dados_wms.shape[0] - df_dados_limpos_wms.shape[0]
        print(f"Dimensões do dataset APÓS remoção de NAs em 'talent6': {df_dados_limpos_wms.shape[0]} linhas, {df_dados_limpos_wms.shape[1]} colunas.")
        print(f"Total de {linhas_removidas_wms} linhas foram removidas devido a valores ausentes em 'talent6'.")

        # O DataFrame 'df_dados_limpos_wms' agora está pronto para as análises subsequentes.
        print("\nProcesso de limpeza da variável 'talent6' no dataset WMS concluído.")
        
        # Visualizar as primeiras linhas do dataset limpo
        print("\nVisualização dos dados do WMS APÓS limpeza (primeiras 5 linhas):\n")
        print(df_dados_limpos_wms.head().to_string())

        # --- Salvando o DataFrame limpo em um novo arquivo Excel ---
        try:
            df_dados_limpos_wms.to_excel(file_path_dados_limpos_excel, sheet_name=sheet_name_dados_limpos, index=False)
            print(f"\nOs dados limpos foram salvos com sucesso em: '{file_path_dados_limpos_excel}' (Aba: '{sheet_name_dados_limpos}')")
        except Exception as e_save:
            print(f"\nERRO ao salvar o arquivo Excel com dados limpos: {e_save}")

    else:
        # Mensagem de erro caso a coluna 'talent6' não seja encontrada no dataset.
        print(f"\nERRO: A coluna 'talent6' não foi encontrada no dataset '{file_path_data_principal_wms}' (Aba: '{sheet_name_data_principal}').")
        print("Verifique o nome da coluna no arquivo original ou no dicionário de variáveis.")
        print(f"Colunas disponíveis no dataset: {df_dados_wms.columns.tolist()}")
        # Em caso de erro, 'df_dados_limpos_wms' pode ser uma cópia do original para evitar quebras em código subsequente,
        # ou o script pode ser interrompido dependendo da criticidade.
        df_dados_limpos_wms = df_dados_wms.copy() # Mantém o DataFrame original se 'talent6' não for encontrado

except FileNotFoundError:
    # Tratamento de erro caso o arquivo de dados principal não seja encontrado.
    print(f"\nERRO CRÍTICO: O arquivo de dados principal '{file_path_data_principal_wms}' não foi encontrado.")
    print("O script não pode prosseguir com a análise dos dados do WMS.")
except ValueError as ve:
    # Tratamento de erro caso a aba especificada não seja encontrada no arquivo Excel
    print(f"\nERRO: A aba '{sheet_name_data_principal}' não foi encontrada no arquivo '{file_path_data_principal_wms}'. Verifique o nome da aba. {ve}")
except Exception as e:
    # Tratamento para outros erros que possam ocorrer durante o carregamento ou processamento dos dados principais.
    print(f"\nERRO CRÍTICO: Ocorreu um erro inesperado ao carregar ou processar o dataset principal '{file_path_data_principal_wms}': {e}")
print("--- Carregamento e Limpeza do Dataset Principal (WMS) Concluídos ---")

# Ao final desta execução, se bem-sucedida, o DataFrame 'df_dados_limpos_wms'
# Conterá os dados do World Management Survey com as linhas onde 'talent6' era ausente removidas,
# E uma tentativa de salvá-lo em um novo arquivo Excel terá sido feita.
# Este DataFrame deve ser usado para as próximas etapas do Trabalho Prático.

--- Iniciando Carregamento do Dicionário de Variáveis ---
Arquivo do dicionário de variáveis 'C:/Users/Inteli/Desktop/VARIABLES_wms.xlsx' (Aba: 'Sheet2') carregado com sucesso.
O dicionário contém 41 variáveis e 3 colunas descritivas.

Visualização inicial do dicionário de variáveis (primeiras 5 linhas):

       variable     type                        information
0        firmid  numeric                     Unique firm ID
1          wave  numeric  Wave when interview was conducted
2       country   string  Country in which plant is located
3           cty   string              2-letter country code
4  i_comptenure  numeric        Manager's tenure in company

AVISO: A coluna 'Variable name' não foi encontrada no dicionário de variáveis.
Colunas disponíveis no dicionário: ['variable', 'type', 'information']
--- Carregamento do Dicionário de Variáveis Concluído ---

--- Iniciando Carregamento e Limpeza do Dataset Principal (WMS) ---
Dataset principal 'C:/Users/Inteli/Desktop/wms_da_textb

In [ ]:
# Passo 3 da Instrução:

